In [ ]:
#BASIC
import numpy as np 
import pandas as pd 
import os

# DATA visualization
import matplotlib.pyplot as plt
import seaborn as sns

import PIL  #https://en.wikipedia.org/wiki/Python_Imaging_Library
from IPython.display import Image, display

from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

import openslide




https://plotly.com/python/plotly-express/#:~:text=Basic%20Charts%20tutorials.-,Overview,for%20creating%20most%20common%20figures.

https://pypi.org/project/plotly-express/

https://github.com/plotly/plotly_express

https://medium.com/plotly/introducing-plotly-express-808df010143d


https://en.wikipedia.org/wiki/Python_Imaging_Library

**Python Imaging Library (PIL)** é uma biblioteca adicional gratuita e de código aberto para a linguagem de programação Python que adiciona suporte para abrir, manipular e salvar muitos formatos de arquivo de imagem diferentes.


In [ ]:
BASE_FOLDER = "/kaggle/input/prostate-cancer-grade-assessment/"
#!ls {BASE_FOLDER}

In [ ]:
mask_dir = f'{BASE_FOLDER}/train_label_masks'

In [ ]:
train = pd.read_csv(BASE_FOLDER+"train.csv")
test = pd.read_csv(BASE_FOLDER+"test.csv")
sub = pd.read_csv(BASE_FOLDER+"sample_submission.csv")

In [ ]:
train.head()

In [ ]:
temp = train.groupby('isup_grade').count()['image_id'].reset_index().sort_values(by='image_id',ascending=False)
temp.style.background_gradient(cmap='Purples')

Now Let's Look at the gleason score distribution as well

In [ ]:
temp = train.groupby('gleason_score').count()['image_id'].reset_index().sort_values(by='image_id',ascending=False)
temp.style.background_gradient(cmap='Reds')

From this graph it is also clear that the data will be baised towards non-cancer examples

In [ ]:
'''
Example for using Openslide to display an image
'''


# Open the image (does not yet read the image into memory)
example = openslide.OpenSlide(os.path.join(BASE_FOLDER+"train_images", '005e66f06bce9c2e49142536caf2f6ee.tiff'))


# Read a specific region of the image starting at upper left coordinate (x=17800, y=19500) on level 0 and extracting a 256*256 pixel patch.
# At this point image data is read from the file and loaded into memory.
patch = example.read_region((17800,19500), 0, (256, 256))


# Display the image
display(patch)

# Close the opened slide after use
example.close()


Agora vamos ver todas as informações que podemos obter de uma imagem depois de criar um objeto Openslide

In [ ]:
train = train.set_index('image_id')
train.head()

In [ ]:
def get_values(image,max_size=(600,400)):
    slide = openslide.OpenSlide(os.path.join(BASE_FOLDER+"train_images", f'{image}.tiff'))
    
    # Here we compute the "pixel spacing": the physical size of a pixel in the image.
    # OpenSlide gives the resolution in centimeters so we convert this to microns.
    f,ax =  plt.subplots(2 ,figsize=(6,16))
    spacing = 1 / (float(slide.properties['tiff.XResolution']) / 10000)
    
    # Read a specific region of the image starting at upper left coordinate (x=17800, y=19500) on level 0 and extracting a 256*256 pixel patch.
    # At this point image data is read from the file and loaded into memory.
    patch = slide.read_region((1780,1950), 0, (256, 256)) #ZOOMED FIGURE
    
    ax[0].imshow(patch) 
    ax[0].set_title('Zoomed Image')
    
    
    ax[1].imshow(slide.get_thumbnail(size=max_size)) #UNZOOMED FIGURE
    ax[1].set_title('Full Image')
    
    
    print(f"File id: {slide}")
    print(f"Dimensions: {slide.dimensions}")
    print(f"Microns per pixel / pixel spacing: {spacing:.3f}")
    print(f"Number of levels in the image: {slide.level_count}")
    print(f"Downsample factor per level: {slide.level_downsamples}")
    print(f"Dimensions of levels: {slide.level_dimensions}\n\n")
    
  
    print(f"ISUP grade: {train.loc[image, 'isup_grade']}")
    print(f"Gleason score: {train.loc[image, 'gleason_score']}")

In [ ]:
get_values('07a7ef0ba3bb0d6564a73f4f3e1c2293')

In [ ]:
get_values('0018ae58b01bdadc8e347995b69f99aa')

In [ ]:
get_values('035b1edd3d1aeeffc77ce5d248a01a53')

In [ ]:
get_values('06a0cbd8fd6320ef1aa6f19342af2e68')

In [ ]:
get_values('0838c82917cd9af681df249264d2769c')

### Agora vamos dar uma olhada nas imagens ampliadas com ISUP e gleason_score diferentes


In [ ]:
def display_images(images):
    '''
   Esta função recebe uma lista de imagens. Em seguida, itera através da imagem criando objetos de slides abertos, nos quais diferentes funções
     para obter informações pode ser chamado mais tarde
    '''
    f, ax = plt.subplots(5,3, figsize=(18,22))
    for i, image in enumerate(images):
        
        slide = openslide.OpenSlide(os.path.join(BASE_FOLDER+"train_images", f'{image}.tiff')) # Making Openslide Object
        
        #Here we compute the "pixel spacing": the physical size of a pixel in the image,
        #OpenSlide gives the resolution in centimeters so we convert this to microns
        
        spacing = 1/(float(slide.properties['tiff.XResolution']) / 10000)
        patch = slide.read_region((1780,1950), 0, (256, 256)) #Reading the image as before betweeen x=1780 to y=1950 and of pixel size =256*256
        
        ax[i//3, i%3].imshow(patch) #Displaying Image
        slide.close()       
        ax[i//3, i%3].axis('off')
        
        image_id = image
        data_provider = train.loc[image, 'data_provider']
        isup_grade = train.loc[image, 'isup_grade']
        gleason_score = train.loc[image, 'gleason_score']
        ax[i//3, i%3].set_title(f"ID: {image_id}\nSource: {data_provider} ISUP: {isup_grade} Gleason: {gleason_score}")

    plt.show() 

In [ ]:
images = [
'07a7ef0ba3bb0d6564a73f4f3e1c2293',
    '037504061b9fba71ef6e24c48c6df44d',
    '035b1edd3d1aeeffc77ce5d248a01a53',
    '059cbf902c5e42972587c8d17d49efed',
    '06a0cbd8fd6320ef1aa6f19342af2e68',
    '06eda4a6faca84e84a781fee2d5f47e1',
    '0a4b7a7499ed55c71033cefb0765e93d',
    '0838c82917cd9af681df249264d2769c',
    '046b35ae95374bfb48cdca8d7c83233f',
    '074c3e01525681a275a42282cd21cbde',
    '05abe25c883d508ecc15b6e857e59f32',
    '05f4e9415af9fdabc19109c980daf5ad',
    '060121a06476ef401d8a21d6567dee6d',
    '068b0e3be4c35ea983f77accf8351cc8',
    '08f055372c7b8a7e1df97c6586542ac8'
]

display_images(images)

OBSERVAÇÕES:

* As dimensões da imagem são bastante grandes (normalmente entre 5.000 e 40.000 pixels em x e y).
* Cada slide tem 3 níveis que você pode carregar, correspondendo a uma redução da resolução de 1, 4 e 16. Os níveis intermediários podem ser criados reduzindo a resolução para um nível de resolução mais alto.
* As dimensões de cada nível diferem com base nas dimensões da imagem original.
* As biópsias podem ser em diferentes rotações. Essa rotação não tem valor clínico e depende apenas de como a biópsia foi coletada no laboratório.
* Existem diferenças de cor perceptíveis entre as biópsias, isso é muito comum na patologia e é causado por diferentes procedimentos laboratoriais.

O que são máscaras?

Além do rótulo no nível do slide (presente no arquivo csv), quase todos os slides do conjunto de treinamento têm uma máscara associada com informações adicionais do rótulo. Essas máscaras indicam diretamente quais partes do tecido são saudáveis ​​e quais são cancerosas. Essas máscaras são fornecidas para ajudar no desenvolvimento de estratégias para selecionar as subamostras mais úteis das imagens.

In [ ]:
example_mask =  openslide.OpenSlide(os.path.join(mask_dir, f'{"00412139e6b04d1e1cee8421f38f6e90"}_mask.tiff'))
display(example_mask.get_thumbnail(size=(600,400)))

Imagem Preta!
As Máscaras para Trem estão no formato RGB, conforme dito pelos organizadores.

Isso acontece pelos dois motivos a seguir:

* As informações do rótulo são armazenadas no canal vermelho (R), os outros canais são * definidos como zero e podem ser ignorados *.

* As máscaras não são dados de imagem como os WSI. São apenas matrizes com valores baseados nas informações do provedor de dados fornecidas acima, em vez de conter uma faixa de valores de 0 a 255, elas vão até no máximo 6, representando o rótulos de classe diferentes (verifique a descrição do conjunto de dados para obter detalhes sobre rótulos de máscara). Portanto, quando você tentar visualizar a máscara, ela aparecerá muito escura, pois todos os valores estão próximos de 0. Aplicar o mapa de cores corrige o problema atribuindo a cada rótulo entre 0 e 6 uma cor distinta.

Então o que precisamos fazer é ler o arquivo de imagem usando o objeto openslide, tirar os valores do Nível Vermelho e então aplicar cmap a ele

Usando uma pequena função auxiliar, podemos exibir algumas informações básicas sobre uma máscara. Para inspecionar as máscaras com mais facilidade, mapeamos os rótulos internos para cores RGB usando uma paleta de cores. Se você preferir algo como matplotlib, você também pode usar plt.imshow () para mostrar diretamente uma máscara (sem convertê-la em uma imagem RGB).

In [ ]:
import matplotlib

def display_masks(slides):    
    f, ax = plt.subplots(5,3, figsize=(18,22))
    for i, slide in enumerate(slides):
        
        mask = openslide.OpenSlide(os.path.join(mask_dir, f'{slide}_mask.tiff'))
        mask_data = mask.read_region((0,0), mask.level_count - 1, mask.level_dimensions[-1])
        cmap = matplotlib.colors.ListedColormap(['black', 'gray', 'green', 'yellow', 'orange', 'red'])

        ax[i//3, i%3].imshow(np.asarray(mask_data)[:,:,0], cmap=cmap, interpolation='nearest', vmin=0, vmax=6) 
        mask.close()       
        ax[i//3, i%3].axis('off')
        
        image_id = slide
        data_provider = train.loc[slide, 'data_provider']
        isup_grade = train.loc[slide, 'isup_grade']
        gleason_score = train.loc[slide, 'gleason_score']
        ax[i//3, i%3].set_title(f"ID: {image_id}\nSource: {data_provider} ISUP: {isup_grade} Gleason: {gleason_score}")
        f.tight_layout()
        
    plt.show()

In [ ]:
display_masks(images) #Visualizing Only six Examples

**Comparando as imagens**



In [ ]:
Aumentando a quantidade de vmax conseguiremos visualizar os perfis aproximados de padrões das células que compõem o tecido.

In [ ]:
def mask_img(image,max_size=(400,400)):
    
    slide = openslide.OpenSlide(os.path.join(BASE_FOLDER+"train_images", f'{image}.tiff'))
    mask =  openslide.OpenSlide(os.path.join(mask_dir, f'{image}_mask.tiff'))
    
    # Here we compute the "pixel spacing": the physical size of a pixel in the image.
    # OpenSlide gives the resolution in centimeters so we convert this to microns.
    f,ax =  plt.subplots(1,2 ,figsize=(18,22))
    spacing = 1 / (float(slide.properties['tiff.XResolution']) / 10000)
    img = slide.get_thumbnail(size=(200,300)) #IMAGE 
    
    mask_data = mask.read_region((0,0), mask.level_count - 1, mask.level_dimensions[-1])
    cmap = matplotlib.colors.ListedColormap(['black', 'gray', 'green', 'yellow', 'orange', 'red'])
    
    ax[0].imshow(img) 
    #ax[0].set_title('Image')
    
    
    ax[1].imshow(np.asarray(mask_data)[:,:,0], cmap=cmap, interpolation='nearest', vmin=0, vmax=4) #IMAGE MASKS
    #ax[1].set_title('Image_MASK')
    
    
    image_id = image
    data_provider = train.loc[image, 'data_provider']
    isup_grade = train.loc[image, 'isup_grade']
    gleason_score = train.loc[image, 'gleason_score']
    ax[0].set_title(f"ID: {image_id}\nSource: {data_provider} ISUP: {isup_grade} Gleason: {gleason_score} IMAGE")
    ax[1].set_title(f"ID: {image_id}\nSource: {data_provider} ISUP: {isup_grade} Gleason: {gleason_score} IMAGE_MASK")

In [ ]:
mask_img('07a7ef0ba3bb0d6564a73f4f3e1c2293')

In [ ]:
images1= [
    '046b35ae95374bfb48cdca8d7c83233f',
    '068b0e3be4c35ea983f77accf8351cc8'
]

for image in images1:
    mask_img(image)

In [ ]:

#pip install -Iv tensorflow_estimator==1.13.0

In [ ]:
 #pip install --upgrade tensorflow

In [ ]:
trainDF #drop([7273],inplace=True)